In [ ]:
import pandas as pd
import numpy as np
from matplotlib import cm, pyplot as plt
from matplotlib.dates import YearLocator, MonthLocator
import cPickle as pk
from hmmlearn.hmm import GaussianHMM
import math

def HMM_MAD(model,obs_levels):
    hidden_states = model.predict(obs_levels)
    means = model.means_.round().astype(int).flatten().tolist()
    predict_levels = np.array([means[state] for state in hidden_states]).reshape(obs_levels.shape)
    abs_error = np.absolute(obs_levels - predict_levels)
    return np.mean(abs_error)/np.mean(obs_levels)

def perc_std_expl(model,obs_levels):
    """
    :param observed: df of observed energy levels per channel
    :param predicted: df of predicted energy levels per channel
    :return: percentage of standard deviation explained
    """
    hidden_states = model.predict(obs_levels)
    means = model.means_.round().astype(int).flatten().tolist()
    predict_levels = np.array([means[state] for state in hidden_states]).reshape(obs_levels.shape)
    obs_mean = np.mean(obs_levels)
    r2 = 1 - (np.sum((obs_levels - predict_levels)**2))/np.sum((obs_levels - obs_mean)**2)
    return 1 - math.sqrt(1-r2)

def r2(model,obs_levels):
    hidden_states = model.predict(obs_levels)
    means = model.means_.round().astype(int).flatten().tolist()
    predict_levels = np.array([means[state] for state in hidden_states]).reshape(obs_levels.shape)
    obs_mean = np.mean(obs_levels)
    r2 = 1 - (np.sum((obs_levels - predict_levels)**2))/np.sum((obs_levels - obs_mean)**2)
    return r2

class HMM():
    def __init__(self, X_test, X_train):
        self.X_train = X_train
        self.X_test = X_test
        self.n_states = None
        self.model =  None

    def fit_HMM(self,error_metric):
        print "Looking for optimal number of states and fitting HMM"
        for i in xrange(2,5):
            candidate = GaussianHMM(n_components=i, covariance_type="full", n_iter=1000)
            candidate.fit(self.X_train)
            if error_metric == HMM_MAD:
                error = HMM_MAD(candidate,self.X_test)
                if i == 2:
                    best_guess = error
                    best_model = candidate
                    opt_n_states = i
                else:
                    if error < best_guess:
                        opt_n_states = i
                        best_model = candidate
                        best_guess = error
            else:
                error = error_metric(candidate,self.X_test)
                if i == 2:
                    best_guess = error
                    best_model = candidate
                    opt_n_states = i
                else:
                    if error > best_guess:
                        opt_n_states = i
                        best_model = candidate
                        best_guess = error
        self.model = best_model
        self.n_states = opt_n_states
        print "Done. Lowest error of {} achieved with {} states".format(best_guess, opt_n_states)

    def extract_means(self):
        return self.model.means_[:,0].flatten()

    def HMM_total_accuracy(self, obs_levels, state_means):
        hidden_states = self.model.predict(obs_levels)
        predict_levels = [state_means[state] for state in hidden_states]
        test_error = 1 - (np.sum(obs_levels[:,0]) - np.sum(predict_levels))/np.sum(obs_levels[:,0])
        return test_error

    def HMM_MAD_perc(self,obs_levels, state_means):
        hidden_states = self.model.predict(obs_levels)
        predict_levels = np.array([state_means[state] for state in hidden_states]).reshape(obs_levels.shape)
        abs_error = np.absolute(obs_levels - predict_levels)
        return np.mean(abs_error)/np.mean(obs_levels)

    def run(self):
        self.fit_HMM()
        state_means = self.extract_means()
        test_error = self.HMM_accuracy(self.X_test, state_means)
        print "Accuracy for the model with {} hidden states is: {}".format(self.n_states,test_error)

In [5]:

import pandas as pd
import numpy as np
import itertools
from hmmlearn.hmm import GaussianHMM
from collections import OrderedDict
from copy import deepcopy
from Preprocessing import cluster

def compute_A_fhmm(list_A):
    result = list_A[0]
    for i in range(len(list_A) - 1):
        result = np.kron(result, list_A[i + 1])
    return result

def combine_parameters(ind_parameters_list):
    '''
    Function to compute factorized HMM parameters such as starting probabilities or transition matrices
    from individual model parameters
    :param ind_parameters_list: parameter list for individual models (per appliance), e.g. list of starting probabilites,
    transition matrices
    :return: matrix of factorized model parameters
    '''
    FactAParam = ind_parameters_list[0]
    for idx in xrange(1,len(ind_parameters_list)):
        FactA = np.kron(FactAParam,ind_parameters_list[idx])
    return FactAParam

def compute_pi_fhmm(list_pi):
    """
    Parameters
    -----------
    list_pi : List of PI's of individual learnt HMMs
    Returns
    -------
    result : Combined Pi for the FHMM
    """
    result = list_pi[0]
    for i in range(len(list_pi) - 1):
        result = np.kron(result, list_pi[i + 1])
    return result

def combine_means(means_list):
    """
    Compute factorized model states means
    :param means_list: list of individual models' state means
    :return: column vector of fatorized state means
    """
    states_combination = list(itertools.product(*means_list))
    num_combinations = len(states_combination)
    means_stacked = np.array([sum(x) for x in states_combination])
    means = np.reshape(means_stacked, (num_combinations, 1))
    cov = np.tile(5 * np.identity(1), (num_combinations, 1, 1))
    return [means, cov]

def sort_startprob(mapping, startprob):
    """ Sort the startprob according to power means; as returned by mapping
    """
    num_elements = len(startprob)
    new_startprob = np.zeros(num_elements)
    for i in xrange(len(startprob)):
        new_startprob[i] = startprob[mapping[i]]
    return new_startprob


def sort_covars(mapping, covars):
    new_covars = np.zeros_like(covars)
    for i in xrange(len(covars)):
        new_covars[i] = covars[mapping[i]]
    return new_covars


def sort_transition_matrix(mapping, A):
    """Sorts the transition matrix according to increasing order of
    power means; as returned by mapping
    Parameters
    ----------
    mapping :
    A : numpy.array of shape (k, k)
        transition matrix
    """
    num_elements = len(A)
    A_new = np.zeros((num_elements, num_elements))
    for i in range(num_elements):
        for j in range(num_elements):
            A_new[i, j] = A[mapping[i], mapping[j]]
    return A_new


def sort_learnt_parameters(startprob, means, covars, transmat):
    mapping = return_sorting_mapping(means)
    means_new = np.sort(means, axis=0)
    startprob_new = sort_startprob(mapping, startprob)
    covars_new = sort_covars(mapping, covars)
    transmat_new = sort_transition_matrix(mapping, transmat)
    assert np.shape(means_new) == np.shape(means)
    assert np.shape(startprob_new) == np.shape(startprob)
    assert np.shape(transmat_new) == np.shape(transmat)

    return [startprob_new, means_new, covars_new, transmat_new]


def return_sorting_mapping(means):
    means_copy = deepcopy(means)
    means_copy = np.sort(means_copy, axis=0)

    # Finding mapping
    mapping = {}
    for i, val in enumerate(means_copy):
        mapping[i] = np.where(val == means)[0][0]
    return mapping

def create_combined_hmm(model):
    list_pi = [model[appliance].startprob_ for appliance in model]
    list_A = [model[appliance].transmat_ for appliance in model]
    list_means = [model[appliance].means_.flatten().tolist()
                  for appliance in model]

    pi_combined = compute_pi_fhmm(list_pi)
    A_combined = compute_A_fhmm(list_A)
    [mean_combined, cov_combined] = combine_means(list_means)

    combined_model = GaussianHMM(
        n_components=len(pi_combined), covariance_type='full',
        startprob_prior=pi_combined, transmat_prior=A_combined)
    combined_model.covars_ = cov_combined
    combined_model.means_ = mean_combined
    combined_model.startprob_ = pi_combined
    combined_model.transmat_ = A_combined
    return combined_model


def decode_hmm(length_sequence, centroids, appliance_list, states):
    """
    Decodes the HMM state sequence
    """
    hmm_states = {}
    hmm_power = {}
    total_num_combinations = 1

    for appliance in appliance_list:
        total_num_combinations *= len(centroids[appliance])

    for appliance in appliance_list:
        hmm_states[appliance] = np.zeros(length_sequence, dtype=np.int)
        hmm_power[appliance] = np.zeros(length_sequence)

    for i in range(length_sequence):

        factor = total_num_combinations
        for appliance in appliance_list:
            # assuming integer division (will cause errors in Python 3x)
            factor = factor // len(centroids[appliance])

            temp = int(states[i]) / factor
            hmm_states[appliance][i] = temp % len(centroids[appliance])
            hmm_power[appliance][i] = centroids[appliance][hmm_states[appliance][i]]
    return [hmm_states, hmm_power]

class FHMM():
    """
    Attributes
    ----------
    model : dict
    predictions : pd.DataFrame()
    meters : list
    MIN_CHUNK_LENGTH : int
    """

    def __init__(self):
        self.model = {}
        self.predictions = pd.DataFrame()
        self.MIN_CHUNK_LENGTH = 100
        self.MODEL_NAME = 'FHMM'


    def train(self, appliances, num_states_dict={}, **load_kwargs):
        """Train using 1d FHMM.
        Places the learnt model in `model` attribute
        The current version performs training ONLY on the first chunk.
        Online HMMs are welcome if someone can contribute :)
        Assumes all pre-processing has been done.
        """
        learnt_model = OrderedDict()
        num_meters = len(appliances)
        if num_meters > 12:
            max_num_clusters = 2
        else:
            max_num_clusters = 3

        for i, app in enumerate(appliances):
            power_data = app.power_data.fillna(value = 0,inplace = False)
            X = power_data.values.reshape((-1, 1))
            assert X.ndim == 2
            self.X = X

            if num_states_dict.get(app.name) is not None:
                # User has specified the number of states for this appliance
                num_total_states = num_states_dict.get(app.name)

            else:
                # Find the optimum number of states
                print "Identifying number of hidden states for appliance {}".format(app.name)
                states = cluster(X, max_num_clusters)
                num_total_states = len(states)
                print "Number of hidden states for appliance {}: {}".format(app.name, num_total_states)

            print("Training model for appliance {} with {} hidden states".format(app.name, num_total_states))
            learnt_model[app.name] = GaussianHMM(num_total_states, "full")

            # Fit
            learnt_model[app.name].fit(X)


        # Combining to make a AFHMM
        self.meters = []
        new_learnt_models = OrderedDict()
        for app in learnt_model:
            startprob, means, covars, transmat = sort_learnt_parameters(
                learnt_model[app].startprob_, learnt_model[app].means_,
                learnt_model[app].covars_, learnt_model[app].transmat_)
            new_learnt_models[app] = GaussianHMM(
                startprob.size, "full")
            new_learnt_models[app].means_ = means
            new_learnt_models[app].covars_ = covars
            new_learnt_models[app].startprob_ = startprob
            new_learnt_models[app].transmat_ = transmat
            # UGLY! But works.
            self.meters.append(app)

        learnt_model_combined = create_combined_hmm(new_learnt_models)
        self.individual = new_learnt_models
        self.model = learnt_model_combined

    def disaggregate_chunk(self, test_mains):
        """Disaggregate the test data according to the model learnt previously
        Performs 1D FHMM disaggregation.
        For now assuming there is no missing data at this stage.
        :param test_mains: test dataframe with aggregate data
        """

        # Array of learnt states
        learnt_states_array = []
        test_mains = test_mains.dropna()
        length = len(test_mains.index)
        temp = test_mains.values.reshape(length, 1)
        learnt_states_array.append(self.model.predict(temp))

        # Model
        means = OrderedDict()
        for elec_meter, model in self.individual.iteritems():
            means[elec_meter] = (
                model.means_.round().astype(int).flatten().tolist())
            means[elec_meter].sort()

        decoded_power_array = []
        decoded_states_array = []

        for learnt_states in learnt_states_array:
            [decoded_states, decoded_power] = decode_hmm(
                len(learnt_states), means, means.keys(), learnt_states)
            decoded_states_array.append(decoded_states)
            decoded_power_array.append(decoded_power)

        prediction = pd.DataFrame(
            decoded_power_array[0], index=test_mains.index)

        return prediction


    def disaggregate(self, mains, output_datastore, sample_period = 20):
        '''Disaggregate mains according to the model learnt previously.
        Parameters
        ----------
        mains : dataframe with aggregated output
        output_datastore : instance of nilmtk.DataStore subclass
            For storing power predictions from disaggregation algorithm.
        sample_period : number, optional
            The desired sample period in minutes.
        **load_kwargs : key word arguments
            Passed to `mains.power_series(**kwargs)`
        '''
        for g, chunk in mains.groupby(np.arange(len(mains)) // sample_period):
            predictions = self.disaggregate_chunk(chunk)
            output_datastore = output_datastore.append(predictions)
            if g != 0 and g%72 == 0:
                print "Disaggregated {} day(s) of data".format(g/72)
        return output_datastore\

In [2]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import pandas as pd
import itertools

class Appliance():
    def __init__(self, name, power_data):
        self.name =  name
        self.power_data = power_data
        self.good_chunks = power_data[(power_data[name]>1)]

def train_test_split(dataframe, split, second_split = None):
    '''
    Splits dataframe into training and validation set
    :param dataframe: total dataframe
    :param split: date on which to split
    :param second_split: option to create second test set
    :return: train, test and optionally second test dataframe
    '''
    df = dataframe.fillna(value = 0,inplace = False)
    df['total'] = dataframe.sum(axis = 1)
    if second_split:
        return df[:split], df[split:second_split], df[second_split:]
    else:
        return df[:split], df[split:]

def create_matrix(appliance,good_chunks = True):
    if not good_chunks:
        power_data = appliance.power_data
    else:
        power_data = appliance.good_chunks
    return power_data.values.reshape((-1, 1))

def cluster(x_train,x_test, max_number_clusters):
    """
    Iteratevely finds an optimal number of clusters based on silhouette score
    :param data: N*K numpy array, in case of a 1D array supply a column vector N*1
    :param max_number_clusters: integer, highest number of clusters
    :return: cluster centers
    """
    highest_score = -1
    for i in xrange(2,max_number_clusters):
        print "Fitting a KMeans model with {} clusters".format(i)
        kmeans = KMeans(n_clusters = i, n_jobs = -1).fit(x_train)
        labels = kmeans.predict(x_test)
        print "Calculating silhouette score..."
        s_score = silhouette_score(x_test, labels, sample_size = 10000)
        if s_score > highest_score:
            highest_score = s_score
            centers = kmeans.cluster_centers_
        print "Silhouette score with {} clusters:{}".format(i,s_score)
    print "Highest silhouete score of {} achieved with {} clusters\n".format(highest_score,len(centers))
    return centers

def Create_combined_states(df):
    new_df = df.copy()
    columns = new_df.columns
    column_combinations = []
    for i in xrange(2,len(columns)+1):
        column_combinations = column_combinations + list(itertools.combinations(columns,i))

    for x in column_combinations:
        name = " ".join(list(x))
        new_df[name] = df[list(x)].sum(axis = 1)
    return new_df